In [3]:
# AusCycling Optimization Problem
# Date: April 24, 2024

# Translate bends into laps
# Print individual cyclist total_work_depleted
# translate 6.95 seconds per 125 m to km/hr
# Code clean up
# Starting and ending cases
# For loop for cyclists order
# Double check negative power corner case

In [8]:
# !pip install gurobipy
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d


In [9]:
# Parameters
constants = pd.read_csv('AusCycling_Constants.csv')
w_prime = pd.read_csv('AusCycling_W_prime.csv')
cp = pd.read_csv('AusCycling_CP.csv')
W_1 = pd.read_csv('AusCycling_W_ij_1.csv')
W_2 = pd.read_csv('AusCycling_W_ij_2.csv')
W_3 = pd.read_csv('AusCycling_W_ij_3.csv')
W_4 = pd.read_csv('AusCycling_W_ij_4.csv')
CdA = pd.read_csv('AusCycling_CdA.csv')

#Lists
bends = list(W_1.index)
cyclists = list(w_prime.index)
n=int(constants.iloc[0])
print(bends)
print(cyclists)
print(n)
print(cp)


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
8
     CP 
0  420.0
1  387.9
2  420.0
3  350.0


C:\Users\salat\AppData\Local\Temp\ipykernel_35184\2203819451.py:14: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  n=int(constants.iloc[0])


In [10]:
#Calculate W_1, W_2, W_3, W_4 matrices to include switch penalties
num_cyclists = 4
drags = [.95, .7, .65, .6]
W_depleted = np.zeros((num_cyclists, len(drags), 32, 32))
critical_power = [420, 388, 420, 350]  # Critical power for each cyclist
w_prime = [29100, 28600, 25500, 26100]  # W prime for each cyclist
raw_power_curves = [
    [1607,1520,1440,1300,1181,1037,926,680,624,591,541,517,501,481,469,460,452,444,432,412],
    [1632,1540,1456,1308,1183,1031,913,653,595,560,507,483,467,447,436,428,420,412,400,380],
    [1489,1411,1338,1212,1104,974,873,651,601,571,526,505,491,473,463,455,448,441,431,412],
    [1462,1380,1305,1173,1061,926,820,589,537,506,459,437,422,404,393,386,379,372,361,343]
]
time = [1,5,10,20,30,45,60,120,150,180,240,300,360,480,600,720,900,1200,2400,3600]
#formulate a num_cyclists x (num_laps * turns_per_lap) power curve array [i,j] 
#to say if cyclist i leads for j turns, this is how much work is depleted
half_lap_time = 6.95
half_lap_distance = 125
penalty_distance = 2.1
penalty_time = (penalty_distance * half_lap_time) / half_lap_distance
#print(penalty_time)
for cyc in range(num_cyclists):
    interpolated_function = interp1d(time, raw_power_curves[cyc], kind = 'cubic')
    for drag in range(len(drags)):
        for col in range(32):
            for row in range(col + 1, 32):
                time_in_lead = (row - col) * half_lap_time + penalty_time
                W_depleted[cyc][drag][row][col]=((interpolated_function(time_in_lead)*drags[drag])-critical_power[cyc]) * time_in_lead
    
#print(W_depleted)

In [1]:
# Create the model object
model= gp.Model ("AusCycling_Model")

# Add the decision variables
z = model.addVars(bends, bends, n, vtype=GRB.BINARY, name="z")

cyclists_init_order = [0,1,2,3]

# Constraints
### Maximum work per cyclist constraint (Need to review this constraint)
max_work_1_constraint = model.addConstr(sum(z[i, j, k] * W_depleted[0, ((cyclists_init_order[0] + (4 - (k % 4))) % 4), i, j] for i in bends for j in bends for k in range(n)) <= 10*w_prime[0], name="max_work_1")
max_work_2_constraint = model.addConstr(sum(z[i, j, k] * W_depleted[1, ((cyclists_init_order[1] + (4 - (k % 4))) % 4), i, j] for i in bends for j in bends for k in range(n)) <= 10*w_prime[1], name="max_work_2")
max_work_3_constraint = model.addConstr(sum(z[i, j, k] * W_depleted[2, ((cyclists_init_order[2] + (4 - (k % 4))) % 4), i, j] for i in bends for j in bends for k in range(n)) <= 10*w_prime[2], name="max_work_3")
max_work_4_constraint = model.addConstr(sum(z[i, j, k] * W_depleted[3, ((cyclists_init_order[3] + (4 - (k % 4))) % 4), i, j] for i in bends for j in bends for k in range(n)) <= 10*w_prime[3], name="max_work_4")

### Number of intervals constraint
number_intervals = model.addConstr(sum(z[i, j, k] for i in bends for j in bends for k in range(n)) == n, name="number_intervals")

### Number of switches per interval constraint
for k in range(n):
    number_switches_per_interval = model.addConstr(sum(z[i, j, k] for i in bends for j in bends) == 1, name="number_switches_in_interval_" + str(k))

interval_start = model.addConstr((sum(z[i,0,0] for i in bends) == 1), name="interval_start")
interval_end = model.addConstr((sum(z[len(bends)-1,j,n-1] for j in bends) == 1), name="interval_end")

# Numer of switches per bend constraint
for i in range(32):
    num_switches_per_bend_1 = model.addConstr(sum(z[i, col, interval] for col in range(32) for interval in range(n)) <= 1, name="num_switches_per_bend_1")
    num_switches_per_bend_2 = model.addConstr(sum(z[row, i, interval] for row in range(32) for interval in range(n)) <= 1, name="num_switches_per_bend_2")

### Intervals start and end from an “Active Bend” 
for s in range(n-1):
    intervals_start_end = model.addConstrs(((sum(z[i, k, s + 1] for i in bends) == sum(z[k, j, s] for j in bends)) for k in bends), name="intervals_start_end")

# Intervals must have bends that go in order
for s in range(n):
    for row in range(32):
        for col in range(row, 32):
            in_order_bends = model.addConstr(z[row, col, s] == 0, name="in_order_bends")
# Objective Function
total_work=(sum(z[i, j, k] * W_depleted[0, ((cyclists_init_order[0] + (4 - (k % 4))) % 4), i, j] for i in bends for j in bends for k in range(n)) + sum(z[i, j, k] * W_depleted[1, ((cyclists_init_order[1] + (4 - (k % 4))) % 4), i, j] for i in bends for j in bends for k in range(n)) + sum(z[i, j, k] * W_depleted[2, ((cyclists_init_order[2] + (4 - (k % 4))) % 4), i, j] for i in bends for j in bends for k in range(n)) + sum(z[i, j, k] * W_depleted[3, ((cyclists_init_order[3] + (4 - (k % 4))) % 4), i, j] for i in bends for j in bends for k in range(n)))
            
objective_function = model.setObjective(total_work, GRB.MAXIMIZE)

model.optimize()


NameError: name 'gp' is not defined

In [13]:
model.ObjVal

388213.6843227841

In [14]:
for k in range(n):
    for j in bends:
        for i in bends:
            # Retrieve the value of the decision variable at indices (i, j, k)
            value = z[i, j, k].X
            if value == 1:
            
                # Print the indices and their corresponding value
                print(f"z[{i}, {j}, {k}] = {value}")

z[4, 0, 0] = 1.0
z[8, 4, 1] = 1.0
z[12, 8, 2] = 1.0
z[16, 12, 3] = 1.0
z[20, 16, 4] = 1.0
z[24, 20, 5] = 1.0
z[28, 24, 6] = 1.0
z[31, 28, 7] = 1.0
